In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore') 

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
sales = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')

In [ ]:
# eksik veri var mı?
print(items.isnull().sum())
print("---------------")
print(sales.isnull().sum())

In [ ]:
# item_id sutun baz alınarak sales ve items birleştirme
df = pd.merge(sales, items, on="item_id")

In [ ]:
# date sütunu gün, ay ve yıl olarak ayırmak
df["date"] = pd.to_datetime(df["date"])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [ ]:
# item_name ve date sütunlarını kaldırmak
df = df.drop(['item_name','date'], axis=1)

In [ ]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()
norm.fit(df.drop('item_cnt_day', axis=1))

In [ ]:
norm_features = norm.transform(df.drop('item_cnt_day',axis=1))
norm_features

# transform = Dönüştürülmüş değerleri olan bir DataFrame üretir. Üretilen DataFrame kendisiyle aynı uzunluğuna sahip olur.

In [ ]:
df_feat = pd.DataFrame(norm_features, columns=df.columns[:-1])

In [ ]:
# Train ve Test oluşturma
x = df_feat
y = df["item_cnt_day"]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3,random_state=1)

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3) # n_neighbors = k
knn.fit(x_train,y_train)
prediction = knn.predict(x_test)
print(" {} nn score: {} ".format(3,knn.score(x_test,y_test)))

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(x_train, y_train) 

print("score: ", gnb.score(x_test,y_test))

### Decison Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)

print("score: ", dt.score(x_test,y_test))

### Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

voting = VotingClassifier(estimators=[("dt",dt),("knn",knn),("gnb",gnb)])

for i in (dt, knn, gnb, voting):
    i.fit(x_train, y_train)
    y_pred = i.predict(x_test)
    print(i, "= ",accuracy_score(y_test, y_pred))

### Değerlendirme Metrikleri: (Voting)

In [ ]:
# Accuracy

from sklearn.metrics import accuracy_score
prediction = voting.predict(x_test)
accuracy = accuracy_score(y_test, prediction)
print(100*accuracy)

In [ ]:
# F-measure

from sklearn.metrics import f1_score
prediction = voting.predict(x_test)

# [None, 'micro', 'macro', 'weighted']
precision = 100*f1_score(y_test, prediction, average=None)
print(precision)

In [ ]:
# Precision

from sklearn.metrics import precision_score
prediction = voting.predict(x_test)

# [None, 'binary' (default), 'micro', 'macro', 'samples', 'weighted']
precision = 100*precision_score(y_test, prediction, average=None) 
print(precision)

In [ ]:
# Recall

from sklearn.metrics import recall_score
prediction = voting.predict(x_test)
precision = 100*recall_score(y_test, prediction, average=None)
print(precision)